# Lexicon Expansion

In [19]:
import nltk
import numpy as np
import sqlite3
from nltk.book import *
from string import punctuation
from collections import Counter,defaultdict
import os
from pprint import pprint
from nltk.corpus import stopwords
sw = stopwords.words('english')
import tweepy
import os
import datetime
import re
from pprint import pprint


In [20]:
#read in the vape file from moodle make sure to encode utf-8

vape_tweets = []

with open("vaping_tweets.csv", encoding = "UTF-8") as input_file:
    next(input_file)
    vape_tweets = [tweet.strip() for tweet in input_file.readlines()]
    

In [21]:
#vape_tweets[:100]

# Building the code step by step for lexicon expansion

In [22]:
## ok I have the file read in now let's pick three words
#hooked too many
#Keep highschool
#Keep ripping
#Keep locker

count = 0

for tweet in vape_tweets :
    if 'locker' in tweet:
        #print(tweet)
        count += 1
#print(count)

In [23]:
# divide the dataset first create key word tweet list and then a non keyword tweet list

keywrd_tweets = []
non_keywrd_tweets = []

for tweet in vape_tweets :
    if 'locker' in tweet or 'ripping' in tweet or 'highschool' in tweet:
           keywrd_tweets.append(tweet)
    else:
        non_keywrd_tweets.append(tweet)

In [24]:
#now let's find words with a high concentration within our keywrd_tweets list

# first let's change our list of tweets to one long string to tokenize / normalize

keywrd_twts = " ".join(keywrd_tweets)

#now lets tokenize/normalize and remove sw's

keywrd_twts = [w.lower() for w in keywrd_twts.split() if w.lower() not in sw and w.isalpha()]
#print(keywrd_twts)

#now let's get the overall length of keywrd_tweet
len(keywrd_twts)



9445

In [25]:
## now let's create our concentration dictionary
concentration_1 = dict()
#first create a freq distribution to get the count of each word in keywrd_tweets

fd1 = nltk.FreqDist(keywrd_twts)
fd1

#now lets get our concentrations

for word,count in fd1.items():
    concentration_1[word]= count/len(keywrd_twts)
    

Now that I have figured out the steps above, I will combine the steps into one large function. I will leave the code below as a jumping off point to test words before throwing them into the function.

In [26]:
#test word counts
#bathroom
#teacher
#principal 

word_set= []

count = 0

for tweet in vape_tweets :
    if 'principal' in tweet:
        #print(tweet)
        count += 1
#print(count)

# Building the Concentration function

In [27]:
# now lets repeat steps 3-5 and add some words to our list

def get_datasets(word1,word2,word3) :
    
    keywrd_tweets = []
    non_keywrd_tweets = []

    for tweet in vape_tweets :
        if (word1) in tweet or (word2) in tweet or (word3) in tweet:
               keywrd_tweets.append(tweet)
        else:
            non_keywrd_tweets.append(tweet)
    
    #now let's find words with a high concentration within our keywrd_tweets list

    # first let's change our list of tweets to one long string to tokenize / normalize

    keywrd_twts = " ".join(keywrd_tweets)

    #now lets tokenize/normalize and remove sw's

    keywrd_twts = [w.lower() for w in keywrd_twts.split() if w.lower() not in sw and w.isalpha()]
    #print(keywrd_twts)

    #now let's get the overall length of keywrd_tweet
    print(len(keywrd_twts))
    
    ## now let's create our concentration dictionary
    concentration_1 = dict()
    #first create a freq distribution to get the count of each word in keywrd_tweets

    fd1 = nltk.FreqDist(keywrd_twts)
    

    #now lets get our concentrations

    for word,count in fd1.items():
        concentration_1[word]= count/len(keywrd_twts)
        
    #sort the dictionary and pick words
    concentration_1 = (sorted(concentration_1.items(), key=lambda item: -1*item[1]))

    return(concentration_1)

# Expanding the wordset

Alright now that I have built my function, it's time to expand my word list in the cells below, I'll repeat the process three times growing my word list to 9.

In [28]:
#first word_set
#get_datasets('highschool','ripping','locker')


In [29]:
#2nd word_set
#get_datasets('pen','kid','highschoolers')

In [30]:
#3rd word_set
#get_datasets('school','nic','hit')

# Final Function Build

Now that we have expanded our wordlist to nine words based on their concentrations. I will write a final function to compare the target tweets data set and the non target tweets dataset. I'll also pull in my get patterns function to get the breakdowns of each.

In [31]:
#The code below quickly turns my two data sets into large strings for get patterns function to work correctly.

keywrd_tweets = []
non_keywrd_tweets = []

for tweet in vape_tweets:
    if 'school' in tweet or 'nic' in tweet or 'hit' in tweet or 'highschoolers' in tweet or 'kid' in tweet or 'pen' in tweet or 'locker' in tweet or 'ripping' in tweet or 'highschool' in tweet:
        keywrd_tweets.append(tweet)
    else:
        non_keywrd_tweets.append(tweet)
        

keywrd_tweets = ' '.join(keywrd_tweets)
non_keywrd_tweets = ' '.join(non_keywrd_tweets)

In [32]:
def get_patterns(text)  :
    """
        This function takes text as an input and returns a dictionary of statistics,
        after cleaning the text. 
    
    """
    
    # We'll make things a big clearer by initializing the 
    # statistics here. These are placeholder values.
    total_tokens = 0
    unique_tokens = 0
    avg_token_len = 0.0
    lex_diversity = 0.0
    top_10 = Counter()
    
    # Do your tokenization and normalization here but in this case I think we will only want to
    
    text = [w.lower() for w in text.split() if w.lower() not in sw and w.isalpha()]
    #text = [w.lower() for w in text if w.lower() not in sw]
    
    
    # Calculate your statistics here
    total_tokens = len(text)
    unique_tokens = len(set(text))
    avg_token_len = sum(len(w) for w in text) / len(text)
    lex_diversity = unique_tokens/total_tokens
    text_count= Counter(text)
    top_10 = text_count.most_common(10)
    
    
    # Now we'll fill out the dictionary. 
    
    results = {'tokens':total_tokens,
               'unique_tokens':unique_tokens,
               'avg_token_length':avg_token_len,
               'lexical_diversity':lex_diversity,
               'top_10':top_10}

    return(results)


In [33]:
#now let's compare the two datasets using our dictionary build from group comparison

def final_comparison(word1,word2,word3,word4,word5,word6,word7,word8,word9,num_words=10,ratio_cutoff=5):
    
    keywrd_tweets = []
    non_keywrd_tweets = []

    for tweet in vape_tweets:
        if word1 in tweet or word2 in tweet or word3 in tweet or word4 in tweet or word5 in tweet or word6 in tweet or word7 in tweet or word8 in tweet or word9 in tweet:
            keywrd_tweets.append(tweet)
        else:
            non_keywrd_tweets.append(tweet)
                
    
    #now let's find words with a high concentration within our keywrd_tweets list

    # first let's change our list of tweets to one long string to tokenize / normalize

    keywrd_twts = " ".join(keywrd_tweets)
    non_keywrd_twts = " ".join(non_keywrd_tweets)

    #now lets tokenize/normalize and remove sw's

    keywrd_twts = [w.lower() for w in keywrd_twts.split() if w.lower() not in sw and w.isalpha()]
    non_keywrd_twts = [w.lower() for w in non_keywrd_twts.split() if w.lower() not in sw and w.isalpha()]
    
    ## now let's create our concentration dictionary FOR BOTH LISTS
    concentration_1 = dict()
    #first create a freq distribution to get the count of each word in keywrd_tweets

    fd1 = nltk.FreqDist(keywrd_twts)
    
    concentration_2 = dict()

    fd2 = nltk.FreqDist(non_keywrd_twts)

    #now lets get our concentrations for both sets of tweets

    for word,count in fd1.items():
        concentration_1[word]= count/len(keywrd_twts)
    
    for word,count in fd2.items():
        concentration_2[word]= count/len(non_keywrd_twts)


    #now let's bring in our comparison dictionary from group comparison and get our stats
    keywrd_twts_vs_non_keywrd_twts = defaultdict()

    for word, concentration1 in concentration_1.items() :

        if fd1[word]> ratio_cutoff and fd2[word] > ratio_cutoff:

            concentration2 = concentration_2[word]

            keywrd_twts_vs_non_keywrd_twts[word]= concentration1/concentration2
    
    
    non_keywrd_twts_vs_keywrd_twts = defaultdict()

    for word, concentration2 in concentration_2.items() :

        if fd2[word]> ratio_cutoff and fd1[word] > ratio_cutoff:

            concentration1 = concentration_1[word]

            non_keywrd_twts_vs_keywrd_twts[word]= concentration2/concentration1
    
    
    results=dict()
    
    results["keywrd_twts_vs_non_keywrd_twts"] = sorted(keywrd_twts_vs_non_keywrd_twts.items(),  key=lambda item: -1*item[1])[:num_words]
    results["non_keywrd_twts_vs_keywrd_twts"] = sorted(non_keywrd_twts_vs_keywrd_twts.items(),  key=lambda item: -1*item[1])[:num_words]
    
    return(results)


# Final Results section

In this section both functions can be used to compare different data sets, and finish out lexicon expansion. Enjoy

In [34]:
final_comparison('school','nic','hit','pen','kid','highschoolers','highschool','ripping','locker')

{'keywrd_twts_vs_non_keywrd_twts': [('divided', 193.12493659558882),
  ('districts', 63.263927452792856),
  ('bryson', 59.62550908989748),
  ('erin', 57.277244177110425),
  ('appropriate', 35.4159188238076),
  ('steven', 32.67151183008082),
  ('contains', 30.586874321519687),
  ('charli', 30.025119371844273),
  ('dab', 24.432608846843046),
  ('leaked', 21.255930018022813)],
 'non_keywrd_twts_vs_keywrd_twts': [('ipad', 84.63030466359538),
  ('premarital', 71.5345794004867),
  ('extramarital', 71.18477705623006),
  ('spotted', 53.51975867126879),
  ('publishes', 47.48566823284143),
  ('grabs', 44.73433825589963),
  ('tiwa', 41.90473083129201),
  ('motivating', 41.01432486409324),
  ('subohm', 35.5284823306066),
  ('storage', 34.38265542089409)]}

In [35]:
get_patterns(keywrd_tweets)

{'tokens': 1437860,
 'unique_tokens': 40350,
 'avg_token_length': 5.563310058002865,
 'lexical_diversity': 0.028062537381942608,
 'top_10': [('juul', 59587),
  ('nicotine', 45649),
  ('vape', 18810),
  ('smoking', 17023),
  ('vaping', 15302),
  ('like', 14951),
  ('kids', 10774),
  ('get', 10296),
  ('cigarettes', 8642),
  ('one', 7620)]}

In [37]:
get_patterns(non_keywrd_tweets)

{'tokens': 2348853,
 'unique_tokens': 59326,
 'avg_token_length': 5.4727400991036905,
 'lexical_diversity': 0.025257434160417872,
 'top_10': [('juul', 67947),
  ('smoking', 52100),
  ('vaping', 41436),
  ('vape', 36903),
  ('new', 25739),
  ('like', 19682),
  ('cigarettes', 17782),
  ('logic', 14770),
  ('aspire', 12420),
  ('get', 12357)]}